In [1]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
import pyspark.sql.functions  as F
from pyspark.sql.types import IntegerType, DateType, TimestampType

In [2]:
config = configparser.ConfigParser()
config.read('dl.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config['AWS'] ['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['AWS']['AWS_SECRET_ACCESS_KEY']

In [3]:
spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()

In [4]:
# get filepath to song data file
song_data = 'data/song_data/*/*/*/*.json'

In [5]:
# read song data file
df = spark.read.json(song_data)

In [6]:
# extract columns to create songs table
song_columns = ['song_id', 'title', 'artist_id', 'year', 'duration']
songs_table = df.select(song_columns).dropDuplicates()

In [7]:
# # write songs table to parquet files partitioned by year and artist
# songs_table.write.partitionBy('year', 'artist_id').parquet('data/songs/')

In [8]:
# extract columns to create artists table
artist_columns = ['artist_id', 'artist_name as name', 'artist_location as location', 'artist_latitude as latitude', 'artist_longitude as longitude']
artists_table = df.selectExpr(artist_columns).dropDuplicates()

In [9]:
# # write artists table to parquet files
# artists_table.write.parquet('data/artists/')

In [10]:
# get filepath to log data file
log_data = 'data/log_data/*.json'

# read log data file
df = spark.read.json(log_data)
df.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



In [11]:
# filter by actions for song plays
df = df.filter(df.page == 'NextSong')

# extract columns for users table  
user_columns = ['userId as user_id', 'firstName as first_name', 'lastName as last_name', 'gender', 'level']
users_table = df.selectExpr(user_columns).dropDuplicates()

In [12]:
# # write users table to parquet files
# users_table.write.parquet('data/users')

In [13]:
# create timestamp column from original timestamp column
# get_timestamp = F.udf(date_convert, TimestampType())
df = df.withColumn('timestamp', F.to_timestamp(F.from_unixtime((F.col('ts') / 1000) , 'yyyy-MM-dd HH:mm:ss.SSS')).cast('timestamp'))

In [14]:
# df.select('ts').show()
df.show(4)

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+-------------------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|          timestamp|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+-------------------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|2018-11-15 00:30:26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|S

In [38]:
df.select('datetime').show()

Py4JJavaError: An error occurred while calling o432.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 156, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark-2.4.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 377, in main
    process()
  File "/opt/spark-2.4.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/worker.py", line 372, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/opt/spark-2.4.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 345, in dump_stream
    self.serializer.dump_stream(self._batched(iterator), stream)
  File "/opt/spark-2.4.3-bin-hadoop2.7/python/lib/pyspark.zip/pyspark/serializers.py", line 141, in dump_stream
    for obj in iterator:
  File "/opt/spark-2.4.3-bin-hadoop2.7/python/lib/pyspark.zip/pysp

In [16]:
# extract columns to create time table
time_table = (
        df.withColumn('hour', F.hour(F.col('timestamp')))
          .withColumn('day', F.dayofmonth(F.col('timestamp')))
          .withColumn('week', F.weekofyear(F.col('timestamp')))
          .withColumn('month', F.month(F.col('timestamp')))
          .withColumn('year', F.year(F.col('timestamp')))
          .withColumn('weekday', F.dayofweek(F.col('timestamp')))
          .select(
            F.col('timestamp').alias('start_time'),
            F.col('hour'),
            F.col('day'),
            F.col('week'),
            F.col('month'),
            F.col('year'),
            F.col('weekday')
          )
    )
  

In [17]:
# # write time table to parquet files partitioned by year and month
# time_table.write.partitionBy('year', 'month').parquet('data/time')

In [18]:
time_table.show(4)

+-------------------+----+---+----+-----+----+-------+
|         start_time|hour|day|week|month|year|weekday|
+-------------------+----+---+----+-----+----+-------+
|2018-11-15 00:30:26|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:41:21|   0| 15|  46|   11|2018|      5|
|2018-11-15 00:45:41|   0| 15|  46|   11|2018|      5|
|2018-11-15 03:44:09|   3| 15|  46|   11|2018|      5|
+-------------------+----+---+----+-----+----+-------+
only showing top 4 rows



In [20]:
# read in song data to use for songplays table
songs_df = spark.read.parquet('data/songs')

artists_df = spark.read.parquet('data/artists')

In [22]:
# extract columns from joined song and log datasets to create songplays table 
songplays_table = (
        df.withColumn('songplay_id', F.monotonically_increasing_id())
          .join(songs_df, songs_df.title == df.song)
          .select(
            'songplay_id', 
            F.col('timestamp').alias('start_time'),
            F.col('userId').alias('user_id'),
            'level',
            'song_id',
            'artist_id',
            F.col('sessionId').alias('session_id'),
            'location',
            F.col('userAgent').alias('user_agent')
          )
    ) 

In [23]:
songplays_table.show(4)

+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|songplay_id|         start_time|user_id|level|           song_id|         artist_id|session_id|            location|          user_agent|
+-----------+-------------------+-------+-----+------------------+------------------+----------+--------------------+--------------------+
|        882|2018-11-21 21:56:47|     15| paid|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|
|        966|2018-11-14 05:06:03|     10| free|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       484|Washington-Arling...|"Mozilla/5.0 (Mac...|
|       4089|2018-11-19 09:14:20|     24| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       672|Lake Havasu City-...|"Mozilla/5.0 (Win...|
|       4473|2018-11-27 22:35:59|     80| paid|SOGDBUF12A8C140FAA|AR558FS1187FB45658|       992|Portland-South Po...|"Mozilla/5.0 (Mac...|
+-----------+--------------

In [25]:
# write songplays table to parquet files partitioned by year and month
songplays_table.write.parquet('data/songplays')